# Laboratorium 5
### Przeszukiwanie wzorca 2d

### 1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego

In [1]:
class Node(object):
    ix=0
    def __init__(self):
        self.next = {}
        self.fail = None
        self.ends = -1
        self.ix = Node.ix
        Node.ix += 1
        
        
    def prin(self, indent=0):
        print("%s%d %s%s"%(' '*indent, self.ix, self.next.keys(), '' if self.ends==-1 else '-'+str(self.ends)))
        if self.fail is not None:
            print("fail ", self.fail.ix)
        for kid in self.next:
            print(kid)
            self.next[kid].prin(indent+1)


class Aho(object):
    def __init__(self):
        Node.ix = 0
        self.__root = Node()
        self.__cols = []
        self.__wid = 0
        self.__hei = 0

        
    def add2dPattern(self, pattern):
        self.__wid = len(pattern[0])
        self.__hei = len(pattern)
        for i in range(len(pattern[0])):
            tmp = self.__root
            for j in range(len(pattern)):
                tmp = tmp.next.setdefault(pattern[j][i], Node())
            tmp.ends=i if tmp.ends==-1 else tmp.ends
            self.__cols.append(tmp.ends)
        

    def make(self):
        tmpQueue = list()
        tmpQueue.append(self.__root)
        while len(tmpQueue) > 0:
            temp = tmpQueue.pop()
            p = None
            for k, v in temp.next.items():
                if temp == self.__root:
                    temp.next[k].fail = self.__root
                else:
                    p = temp.fail
                    while p is not None:
                        if k in p.next:
                            temp.next[k].fail = p.next[k]
                            break
                        p = p.fail
                    if p is None:
                        temp.next[k].fail = self.__root
                tmpQueue.append(temp.next[k])

                
    def fill2d(self, text):
        res = [[0 for i in range(len(text[0]))] for j in range(len(text))]
        for i in range(len(text[0])):
            cur = self.__root
            for j in range(len(text)):
                if text[j][i] not in cur.next:
                    while cur is not self.__root and text[j][i] not in cur.next:
                        cur = cur.fail
                    if text[j][i] in cur.next:
                        cur = cur.next[text[j][i]]
                else:
                    cur = cur.next[text[j][i]]
                res[j][i] = cur.ends
        return res
    
    
    def __find(self, text, le):
        res = []
        for i in range(len(text)):
            cur = self.__root
            for j in range(len(text[0])):
                if text[i][j] not in cur.next:
                    while cur is not self.__root and text[i][j] not in cur.next:
                        cur = cur.fail
                    if text[i][j] in cur.next:
                        cur = cur.next[text[i][j]]
                else:
                    cur = cur.next[text[i][j]]
                if cur.ends != -1:
                    res.append((i, j))
        return res
    
    
    def find(self, text):
        res = self.fill2d(text)
        tmp = Aho()
        pat = [[i] for i in self.__cols]
#         print(pat)
        tmp.add2dPattern(pat)
        tmp.make()
#         tmp.prin()
        wyn = tmp.__find(res, len(self.__cols))
        tot = []
        for x, y in wyn:
            tot.append((x-self.__hei+1, y-self.__wid+1))
        return tot
        
        
    def prin(self):
        print(self.__cols)
        self.__root.prin()

In [2]:
ah = Aho()
ah.add2dPattern(["abcabc","abcacc","abcade"])
ah.make()
# ah.prin()
txt=["pabcasdt","gabcabcx","gabcaccz","gabcadeh","uikmnryt"]
res = ah.fill2d(txt)

In [3]:
print('\n'.join(txt))
res

pabcasdt
gabcabcx
gabcaccz
gabcadeh
uikmnryt


[[-1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1],
 [-1, 0, 1, 2, 0, -1, -1, -1],
 [-1, 0, 1, 2, 0, 4, 5, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1]]

In [4]:
ah.find(txt)

[(1, 1)]

### 2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku.

In [5]:
def find2chars(simple=True):
    with open("haystack.txt", 'r') as file:
        lines = file.readlines()
        res=[]
        if simple:
            for i in range(len(lines)-1):
                for j in range(min(len(lines[i]), len(lines[i+1]))):
                    if ((lines[i][j]>='a' and lines[i][j]<='z') or (lines[i][j]>='A' and lines[i][j]<='Z')) and\
                                lines[i][j]==lines[i+1][j]:
                        res.append(((i,j),(i+1, j)))
            print(len(res))
            f=True
            for a, b in res:
                if lines[a[0]][a[1]] != lines[b[0]][b[1]]:
                    f=False
                    print((a,b), lines[a[0]][a[1]], lines[b[0]][b[1]])
            if f:
                print("OK")
            else:
                print("Not OK")
        else:
            maks=0
            for i in range(len(lines)):
                maks=max(maks, len(lines[i]))
            for i in range(len(lines)):
                lines[i]+='#'*(maks-len(lines[i]))
            
            small = [chr(x) for x in range(ord('a'), ord('z') + 1)]
            great = [chr(x) for x in range(ord('A'), ord('Z') + 1)]
            
            for char in small+great:
                ah = Aho()
                ah.add2dPattern([[char], [char]])
                ah.make()
                res += ah.find(lines)
            
            print(len(res))
            f=True
            for a, b in res:
                if lines[a][b] != lines[a+1][b]:
                    f=False
                    print(((a,b), (a+1,b)), lines[a][b], lines[a+1][b])
            
            if f:
                print("OK")
            else:
                print("Not OK")

In [6]:
find2chars(True)

235
OK


In [7]:
find2chars(False)

235
OK


### 3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji.

In [8]:
def findth():
    with open("haystack.txt", 'r') as file:
        lines = file.readlines()
        res=[]
        maks=0
        for i in range(len(lines)):
            maks=max(maks, len(lines[i]))
        for i in range(len(lines)):
            lines[i]+='#'*(maks-len(lines[i]))

        ah = Aho()
        ah.add2dPattern(['th', 'th'])
        ah.make()
        res += ah.find(lines)
        
        ah = Aho()
        ah.add2dPattern(['t h', 't h'])
        ah.make()
        res += ah.find(lines)

        print(len(res))
        print(res)

In [9]:
findth()

1
[(37, 0)]


### 4. Wybierz przynajmniej 4 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png"

In [10]:
def find4lettxt(letters=['x','q','y','v']):
    with open("haystack.txt", 'r') as file:
#         lines = base64.b64encode(file.read())
        lines = file.readlines()
        tot=[]
        maks=0
#         print(lines)
#         return
        for i in range(len(lines)):
            maks=max(maks, len(lines[i]))
        for i in range(len(lines)):
            lines[i]+='#'*(maks-len(lines[i]))
    
        for char in letters:
            res = []
            ah = Aho()
            ah.add2dPattern([char])
            ah.make()
            res += ah.find(lines)
            tot += res
            print(char, len(res), res)
        print(len(tot), tot)

In [11]:
find4lettxt()

x 57 [(1, 13), (3, 14), (3, 26), (4, 47), (5, 38), (6, 32), (7, 31), (8, 28), (8, 47), (9, 36), (9, 46), (13, 14), (20, 5), (22, 54), (23, 72), (27, 53), (28, 68), (29, 68), (30, 37), (31, 4), (31, 67), (32, 11), (32, 66), (33, 47), (34, 23), (36, 39), (38, 49), (41, 79), (43, 40), (43, 74), (45, 26), (46, 73), (47, 9), (48, 20), (49, 22), (50, 19), (53, 19), (57, 55), (58, 51), (59, 28), (59, 77), (60, 74), (61, 55), (64, 19), (65, 56), (66, 63), (68, 47), (69, 59), (71, 5), (72, 9), (74, 55), (76, 52), (79, 14), (80, 54), (81, 12), (81, 58), (82, 72)]
q 11 [(2, 28), (2, 66), (5, 20), (9, 15), (15, 33), (27, 60), (27, 78), (30, 12), (53, 77), (66, 50), (77, 72)]
y 65 [(0, 39), (0, 78), (1, 24), (1, 57), (3, 64), (4, 58), (4, 71), (5, 63), (6, 58), (7, 27), (8, 42), (8, 78), (9, 39), (11, 60), (11, 69), (13, 41), (14, 63), (15, 39), (16, 1), (18, 56), (21, 68), (21, 77), (23, 76), (24, 51), (24, 67), (25, 70), (30, 31), (36, 14), (36, 46), (40, 45), (42, 34), (42, 79), (43, 18), (43, 5

In [12]:
import base64

In [13]:
def find4letpng(letters=['x','q','y','v']):
    with open("haystack.png", 'br') as file:
        lines = base64.b64encode(file.read())
#         lines = file.readlines()
        tot=[]
        maks=0
#         for i in range(len(lines)):
#             maks=max(maks, len(lines[i]))
#         for i in range(len(lines)):
#             lines[i]+='#'*(maks-len(lines[i]))
    
        for char in letters:
            res = []
            ah = Aho()
            ah.add2dPattern([char])
            ah.make()
            res += ah.find([lines])
            tot += res
            print(char, len(res), res)
        print(len(tot), tot)

In [14]:
find4letpng()

x 0 []
q 0 []
y 0 []
v 0 []
0 []


In [15]:
from PIL import Image
import numpy as np

In [16]:
def find4letpngobrazek(letters=['x', 'q', 'y', 'v']):
    img = Image.open('haystack.png')
    img = img.convert('1')
    haystack = np.array(img).tolist()
    for letter in letters:
        res = []
        let = Image.open(letter+'.png').convert('1')
        let = np.array(let).tolist()
        ah = Aho()
        ah.add2dPattern(let)
        ah.make()
        res += ah.find(haystack)
        print(letter, len(res), res)

In [17]:
find4letpngobrazek()

x 50 [(102, 139), (102, 238), (124, 419), (146, 342), (168, 297), (190, 314), (212, 254), (212, 410), (234, 366), (234, 447), (322, 146), (476, 70), (520, 486), (630, 464), (696, 361), (718, 63), (718, 596), (740, 120), (740, 602), (762, 421), (784, 241), (828, 372), (872, 458), (938, 708), (982, 660), (1026, 272), (1070, 112), (1092, 212), (1114, 210), (1136, 175), (1202, 195), (1290, 521), (1312, 474), (1334, 266), (1334, 704), (1356, 675), (1378, 515), (1444, 195), (1466, 537), (1488, 573), (1532, 441), (1554, 561), (1598, 75), (1620, 101), (1664, 519), (1708, 489), (1774, 156), (1818, 120), (1818, 566), (1840, 660)]
q 1 [(630, 517)]
y 4 [(190, 278), (212, 678), (498, 710), (960, 339)]
v 8 [(124, 53), (190, 689), (740, 310), (806, 503), (1202, 345), (1554, 123), (1576, 535), (1818, 248)]


Niestety nie znajduje wszystkich możliwości, prawdopodobnie jest to spowodowane nienajlepiej wyciętymi obrazkami(chciałem zostawić białą otoczkę pixeli dookoła całego znaku), możliwe że znaki z przodu, tyłu u góry lub u dołu, mogą nachodzić na fragment i stąd wynikają niezgodności.

In [18]:
find4letpngobrazek(['x1', 'q1', 'y1', 'v1'])

x1 0 []
q1 1 [(631, 518)]
y1 12 [(37, 357), (103, 572), (125, 516), (191, 279), (213, 679), (235, 389), (389, 34), (499, 711), (961, 340), (983, 166), (1005, 524), (1247, 34)]
v1 8 [(125, 54), (191, 690), (741, 311), (807, 504), (1203, 346), (1555, 124), (1577, 536), (1819, 249)]


W niektórych przypadkach obcięcie obwódki nie zaszkodziło, a nawet pomogło, jednak z jakiegoś powodu przy x przeszkodziło, może to być spowodowane modyfikacją i uszkodzeniem pliku podczas obróki w programie graficznym.

### 5. Znajdź wszystkie wystąpienia słowa "p a t t e r n" w haystack.png.

In [19]:
def findpattxt(pattern=['p a t t e r n']):
    with open("haystack.txt", 'r') as file:
        lines = file.readlines()
        res=[]
        maks=0
        for i in range(len(lines)):
            maks=max(maks, len(lines[i]))
        for i in range(len(lines)):
            lines[i]+='#'*(maks-len(lines[i]))
        
        ah = Aho()
        ah.add2dPattern(pattern)
        ah.make()
        res += ah.find(lines)
        print(len(res), res)

In [20]:
findpattxt()

5 [(20, 17), (21, 44), (23, 25), (25, 13), (27, 25)]


In [21]:
def findpatpng(pattern=['p a t t e r n']):
    with open("haystack.png", 'rb') as file:
        lines = base64.b64encode(file.read())
#         lines = file.readlines()
        res=[]
        maks=0
        
        ah = Aho()
        ah.add2dPattern(pattern)
        ah.make()
        res += ah.find([lines])
        print(len(res), res)

In [22]:
findpatpng()

0 []


In [23]:
def find4patpngobrazek(file='pattern'):
    img = Image.open('haystack.png')
    img = img.convert('1')
    haystack = np.array(img).tolist()
    res = []
    let = Image.open(file+'.png').convert('1')
    let = np.array(let).tolist()
    ah = Aho()
    ah.add2dPattern(let)
    ah.make()
    res += ah.find(haystack)
    print(file, len(res), res)

In [24]:
find4patpngobrazek()

pattern 1 [(627, 247)]


Niestety nie znajduje wszystkich możliwości, prawdopodobnie jest to spowodowane nienajlepiej wyciętymi obrazkami(chciałem zostawić białą otoczkę pixeli dookoła całego 'p a t t e r n'), możliwe że znaki z przodu, tyłu u góry lub u dołu, mogą nachodzić na fragment i stąd wynikają niezgodności.

In [25]:
find4patpngobrazek("patt")

patt 0 []


Obcięcie otoczki w tym przypadku z jakiegoś powodu zaszkodziło, możliwe że jakieś uszkodzenie wynikające z obcinania obrazka narzędziem graficznym.

### 6. Porównaj czas budowania automatu i czas wyszukiwania dla różnych rozmiarów wzorca

In [26]:
import timeit

In [27]:
def test():
    with open("haystack.txt", 'r') as file:
        lines = file.readlines()
        res=[]
        maks=0
        for i in range(len(lines)):
            maks=max(maks, len(lines[i]))
        for i in range(len(lines)):
            lines[i]+='#'*(maks-len(lines[i]))
        
        text1 = ["T h e search of words or p a t t e r n s in static texts is quite a",
                 "t h a n the previous pattern-matching mechanism. Dictionaries, for ",
                 "are organized in order to speed u p the access to entries. Another ",
                 "of the same question is given by indexes. Technical books often con",
                 "index of chosen terms t h a t gives pointers to p a r t s of the te",
                 "in the index. T h e algorithms involved in the creation of an index",
                 "specific group. T h e use of dictionaries or lexicons is often rela",
                 "language processing. Lexicons of programming languages are small, a",
                 "representation is not a difficult problem during the development of",
                 "To the contrary, English contains approximately 100,000 words, a n ",
                 "t h a t if inflected forms are considered. In French, inflected for",
                 "t h a n 700,000 words. T h e representation of lexicons of this siz"]
        text2 = ["T H e", "t h a"]
        
        print("Making longer pattern")
        ah = Aho()
        %timeit index = ah.add2dPattern(text1)
        %timeit index = ah.make()
        print("Finidng longer pattern")
        %timeit index = ah.find(lines)
        
        print("Making shorter pattern")
        ah = Aho()
        %timeit index = ah.add2dPattern(text2)
        %timeit index = ah.make()
        print("Finding shorter pattern")
        %timeit index = ah.find(lines)
        

In [28]:
test()

Making longer pattern
886 µs ± 7.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
729 µs ± 9.08 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Finidng longer pattern
1.29 s ± 9.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Making shorter pattern
14.8 µs ± 145 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.5 µs ± 63.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Finding shorter pattern
10.1 s ± 501 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Jak widać krótszy wzorzec robi się dużo krócej, jednak jest wyszukiwany dłużej

### 7. Podziel plik na 2, 4 i 8 fragmentów (w poziomie) i porównaj czas przeszukiwania

In [29]:
def testparts():
    with open("haystack.txt", 'r') as file:
        lines = file.readlines()
        res=[]
        maks=0
        for i in range(len(lines)):
            maks=max(maks, len(lines[i]))
        for i in range(len(lines)):
            lines[i]+='#'*(maks-len(lines[i]))
        
        text  = ["T h e search of words or p a t t e r n s in static texts is quite a",
                 "t h a n the previous pattern-matching mechanism. Dictionaries, for ",
                 "are organized in order to speed u p the access to entries. Another ",
                 "of the same question is given by indexes. Technical books often con",
                 "index of chosen terms t h a t gives pointers to p a r t s of the te",
                 "in the index. T h e algorithms involved in the creation of an index"]
        
        print("Making pattern")
        ah = Aho()
        %timeit index = ah.add2dPattern(text1)
        %timeit index = ah.make()
        print("Finidng:")
        %timeit index = ah.find(lines)
        print("Finding in halves:")
        %timeit index = ah.find(lines[:len(lines)//2])
        %timeit index = ah.find(lines[len(lines)//2:])
        print("Finding in quarters:")
        %timeit index = ah.find(lines[:len(lines)//4])
        %timeit index = ah.find(lines[len(lines)//4:len(lines)//2])
        %timeit index = ah.find(lines[len(lines)//2:3*len(lines)//4])
        %timeit index = ah.find(lines[3*len(lines)//4:])
        print("Finding in eighths:")
        for i in range(8):
            %timeit index = ah.find(lines[i*len(lines)//8:(i+1)*len(lines)//8])

In [30]:
testparts()

Making pattern
917 µs ± 7.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
733 µs ± 8.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Finidng:
1.47 s ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Finding in halves:
1.65 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.43 s ± 73.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Finding in quarters:
1.35 s ± 50 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.3 s ± 46.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.34 s ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.28 s ± 3.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Finding in eighths:
1.29 s ± 8.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.28 s ± 5.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.29 s ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.28 s ± 5.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.3 s ± 48 ms per loop (mean ±

Jak widać długość tekstu przeszukiwanego z jakiegoś powodu ma mały wpływ długość przeszukiwania.